In [1]:
import torch 
import torch.nn as nn 
import numpy as np 
from torch.utils import data
import torch.nn.functional as F

%matplotlib inline

In [2]:
data_path = 'data/nietzsche.txt'
text = open(data_path).read()

In [3]:
len(text)

600893

In [4]:
vocab = sorted(list(set(text)))
vocab.insert(0, '\0')
vocab_size = len(vocab)

In [5]:
char_indices = {v:k for k,v in enumerate(vocab)} 
indices_char = {k:v for k,v in enumerate(vocab)}

In [6]:
idx = [char_indices[c] for c in text]

In [7]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [8]:
text[:10]

'PREFACE\n\n\n'

## Modify data for 3 hidden layer

In [9]:
c1 = [idx[i] for i in range(len(idx) - 3)]
c2 = [idx[i+1] for i in range(len(idx)-3)]
c3 = [idx[i+2] for i in range(len(idx) -3)]
c4 = [idx[i+3] for i in range(len(idx) -3)]

In [10]:
len(c1), len(c2), len(c3), len(c4)

(600890, 600890, 600890, 600890)

In [11]:
x = np.stack([c1, c2, c3], axis=1)

In [12]:
[i[0] for i in x.T]

[40, 42, 29]

In [13]:
[c1[:10], c2[:10], c3[:10]]

[[40, 42, 29, 30, 25, 27, 29, 1, 1, 1],
 [42, 29, 30, 25, 27, 29, 1, 1, 1, 43],
 [29, 30, 25, 27, 29, 1, 1, 1, 43, 45]]

In [28]:
class textDataset(data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = np.array(x).T
        self.y = np.array(y)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return [i[idx] for i in self.x], self.y[idx]
    
    @classmethod
    def split(cls, x, y, ratio=0.90):
        b = np.random.uniform(len(x)) < 0.90
        train_x = x[b]
        train_y = y[b]
        valid_x = x[~b]
        valid_y = y[~b]
        
        return cls(train_x, train_y), cls(valid_x, valid_y)
        

In [29]:
dataset = textDataset(np.stack([c1, c2, c3], axis=1),c4)

In [32]:
train, valid = textDataset.split(np.stack([c1, c2, c3], axis=1),c4)

In [33]:
loader = data.DataLoader(dataset, batch_size=512)
valid_loader = data.DataLoader(valid, batch_size=512)



## Define model

In [17]:
class CharModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.l_in = nn.Linear(embedding_dim, hidden_dim)
        self.l_hidden = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, vocab_size)
        
    def forward(self, inp):
        in1, in2, in3 = inp
        in1 = F.relu(self.l_in(self.emb(in1)))
        in2 = F.relu(self.l_in(self.emb(in2)))
        in3 = F.relu(self.l_in(self.emb(in3)))
        
        
        # Defined empty hidden layer 
        h = torch.zeros(in1.size())
        h = F.tanh(self.l_hidden(h + in1.type(torch.FloatTensor)))
        h = F.tanh(self.l_hidden(h + in2.type(torch.FloatTensor)))
        h = F.tanh(self.l_hidden(h + in3.type(torch.FloatTensor)))
        return F.log_softmax(self.out(h), dim=1)

In [18]:
model = CharModel(vocab_size, 50, 100)

In [19]:
model

CharModel(
  (emb): Embedding(85, 50)
  (l_in): Linear(in_features=50, out_features=100, bias=True)
  (l_hidden): Linear(in_features=100, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=85, bias=True)
)

In [20]:
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
import ignite

In [22]:
ignite.handlers

<module 'ignite.handlers' from '/Users/rajat/Documents/env/lib/python3.6/site-packages/ignite/handlers/__init__.py'>

In [23]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import CategoricalAccuracy, Loss

In [24]:
trainer = create_supervised_trainer(model, optim, F.nll_loss)

In [25]:
trainer.run(loader, max_epochs=10)

/Users/rajat/Documents/env/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [34]:
import visdom

In [38]:
def create_plot_windows(vis, xlabel, ylabel, title):
    return vis.line(X=np.array([1]), Y=np.array([np.nan]), 
                   opts=dict(xlabel=xlabel, y=ylabel, title=title))

In [45]:
def run(loader, valid_loader, max_epochs, log_interval):
    vis = visdom.Visdom()
    trainer = create_supervised_trainer(model, optim, F.nll_loss)
    evaluator = create_supervised_evaluator(model, {'nll': Loss(F.nll_loss)})
    
    train_loss_window = create_plot_windows(vis, '#iterations', 'Loss', 'Training Loss')
    train_avg_loss_window = create_plot_windows(vis, '#iterations', 'Loss', 'Training avg loss')

    val_loss_window = create_plot_windows(vis, '#iterations', 'Loss', 'validation Loss')

    
    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        iter = (engine.state.iteration - 1) % len(loader) + 1
        if iter % log_interval == 0:
            print("Epoch[{}] Iteration[{}/{}] Loss {:.2f}".format(engine.state.epoch,
                                                                 iter, len(loader), engine.state.output))
            
            vis.line(X=np.array([engine.state.iteration]),
                    Y = np.array([engine.state.output]),
                    update='append', win=train_loss_window)
            
    
    trainer.run(loader, max_epochs=max_epochs)
    


In [47]:
run(loader, valid_loader, 10, 3)

/Users/rajat/Documents/env/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch[1] Iteration[3/1174] Loss 1.88
Epoch[1] Iteration[6/1174] Loss 2.11
Epoch[1] Iteration[9/1174] Loss 2.19
Epoch[1] Iteration[12/1174] Loss 2.08
Epoch[1] Iteration[15/1174] Loss 1.78
Epoch[1] Iteration[18/1174] Loss 1.81
Epoch[1] Iteration[21/1174] Loss 1.92
Epoch[1] Iteration[24/1174] Loss 2.10
Epoch[1] Iteration[27/1174] Loss 1.86
Epoch[1] Iteration[30/1174] Loss 2.29
Epoch[1] Iteration[33/1174] Loss 1.97
Epoch[1] Iteration[36/1174] Loss 1.79
Epoch[1] Iteration[39/1174] Loss 1.91
Epoch[1] Iteration[42/1174] Loss 1.99
Epoch[1] Iteration[45/1174] Loss 1.88
Epoch[1] Iteration[48/1174] Loss 1.78
Epoch[1] Iteration[51/1174] Loss 1.99
Epoch[1] Iteration[54/1174] Loss 1.86
Epoch[1] Iteration[57/1174] Loss 2.02
Epoch[1] Iteration[60/1174] Loss 1.87
Epoch[1] Iteration[63/1174] Loss 1.87
Epoch[1] Iteration[66/1174] Loss 1.88
Epoch[1] Iteration[69/1174] Loss 1.74
Epoch[1] Iteration[72/1174] Loss 1.75
Epoch[1] Iteration[75/1174] Loss 1.83
Epoch[1] Iteration[78/1174] Loss 1.88
Epoch[1] Iterat

Epoch[1] Iteration[642/1174] Loss 1.88
Epoch[1] Iteration[645/1174] Loss 2.05
Epoch[1] Iteration[648/1174] Loss 1.78
Epoch[1] Iteration[651/1174] Loss 2.01
Epoch[1] Iteration[654/1174] Loss 1.95
Epoch[1] Iteration[657/1174] Loss 1.92
Epoch[1] Iteration[660/1174] Loss 1.91
Epoch[1] Iteration[663/1174] Loss 1.92
Epoch[1] Iteration[666/1174] Loss 1.98
Epoch[1] Iteration[669/1174] Loss 2.20
Epoch[1] Iteration[672/1174] Loss 1.79
Epoch[1] Iteration[675/1174] Loss 1.80
Epoch[1] Iteration[678/1174] Loss 1.92
Epoch[1] Iteration[681/1174] Loss 1.87
Epoch[1] Iteration[684/1174] Loss 1.87
Epoch[1] Iteration[687/1174] Loss 2.01
Epoch[1] Iteration[690/1174] Loss 2.02
Epoch[1] Iteration[693/1174] Loss 1.94
Epoch[1] Iteration[696/1174] Loss 1.94
Epoch[1] Iteration[699/1174] Loss 2.04
Epoch[1] Iteration[702/1174] Loss 2.18
Epoch[1] Iteration[705/1174] Loss 1.96
Epoch[1] Iteration[708/1174] Loss 1.95
Epoch[1] Iteration[711/1174] Loss 1.87
Epoch[1] Iteration[714/1174] Loss 1.85
Epoch[1] Iteration[717/11

Epoch[2] Iteration[111/1174] Loss 1.96
Epoch[2] Iteration[114/1174] Loss 2.21
Epoch[2] Iteration[117/1174] Loss 1.94
Epoch[2] Iteration[120/1174] Loss 2.02
Epoch[2] Iteration[123/1174] Loss 1.82
Epoch[2] Iteration[126/1174] Loss 2.01
Epoch[2] Iteration[129/1174] Loss 1.94
Epoch[2] Iteration[132/1174] Loss 1.82
Epoch[2] Iteration[135/1174] Loss 1.66
Epoch[2] Iteration[138/1174] Loss 1.83
Epoch[2] Iteration[141/1174] Loss 2.01
Epoch[2] Iteration[144/1174] Loss 2.06
Epoch[2] Iteration[147/1174] Loss 1.98
Epoch[2] Iteration[150/1174] Loss 1.95
Epoch[2] Iteration[153/1174] Loss 1.93
Epoch[2] Iteration[156/1174] Loss 2.05
Epoch[2] Iteration[159/1174] Loss 1.86
Epoch[2] Iteration[162/1174] Loss 1.86
Epoch[2] Iteration[165/1174] Loss 1.78
Epoch[2] Iteration[168/1174] Loss 2.00
Epoch[2] Iteration[171/1174] Loss 1.93
Epoch[2] Iteration[174/1174] Loss 1.86
Epoch[2] Iteration[177/1174] Loss 1.87
Epoch[2] Iteration[180/1174] Loss 2.03
Epoch[2] Iteration[183/1174] Loss 2.11
Epoch[2] Iteration[186/11

Epoch[2] Iteration[756/1174] Loss 1.93
Epoch[2] Iteration[759/1174] Loss 1.90
Epoch[2] Iteration[762/1174] Loss 1.87
Epoch[2] Iteration[765/1174] Loss 2.00
Epoch[2] Iteration[768/1174] Loss 1.97
Epoch[2] Iteration[771/1174] Loss 2.06
Epoch[2] Iteration[774/1174] Loss 1.91
Epoch[2] Iteration[777/1174] Loss 2.09
Epoch[2] Iteration[780/1174] Loss 2.17
Epoch[2] Iteration[783/1174] Loss 1.80
Epoch[2] Iteration[786/1174] Loss 1.78
Epoch[2] Iteration[789/1174] Loss 1.88
Epoch[2] Iteration[792/1174] Loss 1.78
Epoch[2] Iteration[795/1174] Loss 2.01
Epoch[2] Iteration[798/1174] Loss 1.78
Epoch[2] Iteration[801/1174] Loss 1.97
Epoch[2] Iteration[804/1174] Loss 1.93
Epoch[2] Iteration[807/1174] Loss 1.95
Epoch[2] Iteration[810/1174] Loss 1.94
Epoch[2] Iteration[813/1174] Loss 1.85
Epoch[2] Iteration[816/1174] Loss 1.89
Epoch[2] Iteration[819/1174] Loss 1.96
Epoch[2] Iteration[822/1174] Loss 1.75
Epoch[2] Iteration[825/1174] Loss 1.89
Epoch[2] Iteration[828/1174] Loss 1.66
Epoch[2] Iteration[831/11

...

Epoch[6] Iteration[1155/1174] Loss 1.79
Epoch[6] Iteration[1158/1174] Loss 1.84
Epoch[6] Iteration[1161/1174] Loss 1.68
Epoch[6] Iteration[1164/1174] Loss 2.06
Epoch[6] Iteration[1167/1174] Loss 1.86
Epoch[6] Iteration[1170/1174] Loss 1.86
Epoch[6] Iteration[1173/1174] Loss 1.86
Epoch[7] Iteration[3/1174] Loss 1.97
Epoch[7] Iteration[6/1174] Loss 2.22
Epoch[7] Iteration[9/1174] Loss 2.60
Epoch[7] Iteration[12/1174] Loss 2.11
Epoch[7] Iteration[15/1174] Loss 1.85
Epoch[7] Iteration[18/1174] Loss 1.86
Epoch[7] Iteration[21/1174] Loss 2.07
Epoch[7] Iteration[24/1174] Loss 2.16
Epoch[7] Iteration[27/1174] Loss 1.84
Epoch[7] Iteration[30/1174] Loss 2.45
Epoch[7] Iteration[33/1174] Loss 2.03
Epoch[7] Iteration[36/1174] Loss 1.81
Epoch[7] Iteration[39/1174] Loss 1.92
Epoch[7] Iteration[42/1174] Loss 1.99
Epoch[7] Iteration[45/1174] Loss 1.93
Epoch[7] Iteration[48/1174] Loss 1.83
Epoch[7] Iteration[51/1174] Loss 2.02
Epoch[7] Iteration[54/1174] Loss 1.90
Epoch[7] Iteration[57/1174] Loss 2.05
E

Traceback (most recent call last):
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/visdom/__init__.py", line 446, in _send
    data=json.dumps(msg),
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/requests/api.py", line 112, in post
    return request('post', url, data=data, json=json, **kwargs)
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/requests/sessions.py", line 512, in request
    resp = self.send(prep, **send_kwargs)
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/requests/sessions.py", line 622, in send
    r = adapter.send(request, **kwargs)
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/requests/adapters.py", line 445, in send
    timeout=timeout
  File "/Users/rajat/Documents/env/lib/python3.6/site-packages/urllib3/connectionpool.py", lin

Epoch[7] Iteration[504/1174] Loss 1.99
Epoch[7] Iteration[507/1174] Loss 2.02
Epoch[7] Iteration[510/1174] Loss 1.92
Epoch[7] Iteration[513/1174] Loss 1.99
Epoch[7] Iteration[516/1174] Loss 2.04
Epoch[7] Iteration[519/1174] Loss 1.86
Epoch[7] Iteration[522/1174] Loss 2.02
Epoch[7] Iteration[525/1174] Loss 2.03
Epoch[7] Iteration[528/1174] Loss 1.98
Epoch[7] Iteration[531/1174] Loss 1.99
Epoch[7] Iteration[534/1174] Loss 2.05
Epoch[7] Iteration[537/1174] Loss 2.06
Epoch[7] Iteration[540/1174] Loss 1.85
Epoch[7] Iteration[543/1174] Loss 1.86
Epoch[7] Iteration[546/1174] Loss 1.97
Epoch[7] Iteration[549/1174] Loss 2.03
Epoch[7] Iteration[552/1174] Loss 2.15
Epoch[7] Iteration[555/1174] Loss 2.15
Epoch[7] Iteration[558/1174] Loss 2.16
Epoch[7] Iteration[561/1174] Loss 1.96
Epoch[7] Iteration[564/1174] Loss 1.94
Epoch[7] Iteration[567/1174] Loss 1.87
Epoch[7] Iteration[570/1174] Loss 1.84
Epoch[7] Iteration[573/1174] Loss 1.96
Epoch[7] Iteration[576/1174] Loss 1.99
Epoch[7] Iteration[579/11

KeyboardInterrupt: 

Connection is already closed.
